## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Rikitea,PF,-23.1203,-134.9692,79.12,70,2,15.86,light rain,2022-02-12 20:48:04
1,1,Quatre Cocos,MU,-20.2078,57.7625,80.71,86,10,11.18,light rain,2022-02-12 20:48:04
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,-27.40,68,40,6.91,scattered clouds,2022-02-12 20:48:05
3,3,Kruisfontein,ZA,-34.0033,24.7314,72.10,91,95,9.64,overcast clouds,2022-02-12 20:48:05
4,4,Mahebourg,MU,-20.4081,57.7000,80.89,83,40,9.22,scattered clouds,2022-02-12 20:48:05


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What minimum temperature you would like on your trip in Fahrenheit? "))
max_temp = float(input("What maximum temperature you would like on your trip in Fahrenheit? "))

What minimum temperature you would like on your trip in Fahrenheit? 73
What maximum temperature you would like on your trip in Fahrenheit? 77


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
8,8,Lira,UG,2.2350,32.9097,74.48,41,81,2.82,broken clouds,2022-02-12 20:48:07
11,11,Sibolga,ID,1.7427,98.7792,75.25,85,92,2.51,overcast clouds,2022-02-12 20:48:07
12,12,Port Alfred,ZA,-33.5906,26.8910,75.31,93,4,11.25,clear sky,2022-02-12 20:48:08
14,14,Padang,ID,-0.9492,100.3543,76.21,81,96,4.16,overcast clouds,2022-02-12 20:48:08
18,18,Bandarbeyla,SO,9.4942,50.8122,75.96,69,100,13.31,overcast clouds,2022-02-12 20:48:09
45,45,Castro,BR,-24.7911,-50.0119,73.26,67,57,3.91,broken clouds,2022-02-12 20:54:08
49,49,East London,ZA,-33.0153,27.9116,76.14,89,0,9.89,clear sky,2022-02-12 20:54:09
57,57,Bengkulu,ID,-3.8004,102.2655,75.31,84,93,4.70,overcast clouds,2022-02-12 20:49:17
63,63,Luderitz,NaN,-26.6481,15.1594,73.76,69,4,6.44,clear sky,2022-02-12 20:54:13
74,74,Dingle,PH,10.9995,122.6711,74.01,94,40,7.43,scattered clouds,2022-02-12 20:54:15


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))